In [150]:
# MERRA2 DATASET INTERPOLATION
# Subhatra Sivam, Eli Lichtblau

# import packages
from math import radians
import netCDF4 as nc
import numpy as np
import pandas as pd
from typing import Tuple
import time

# functions
# pretty ones from Eli
def solution(X1: np.ndarray, X2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Params:
        X1: (1D array_like)
        X2: (1D array_like)
    Returns:
        X1_indices where value exists in X2 as well
        X2_indices where value exists in X1 as well
    Note: the returned indices array are ordered smallest to greatest. by the value they correspond to
    that is to say X1[X1_indices] is a sorted list, u could do X1[X1_indices.sort()] to get the values in 
    the order they appear in the orignal X1
    
    """
    inter = np.intersect1d(X1, X2)
    def helper(inter: np.ndarray, x: np.ndarray):
        sorter = np.argsort(x)
        searchsorted_left = np.searchsorted(x, inter, sorter=sorter,side='left')
        searchsorted_right = np.searchsorted(x, inter, sorter=sorter,side='right')
        values = vrange(searchsorted_left, searchsorted_right) 
        return sorter[values] # optional to sort this if u care?
        

    return helper(inter, X1), helper(inter, X2)
def vrange(starts: np.ndarray, stops: np.ndarray):
    """Create concatenated ranges of integers for multiple start/stop

    Parameters:
        starts (1-D array_like): starts for each range
        stops (1-D array_like): stops for each range (same shape as starts)

    Returns:
        numpy.ndarray: concatenated ranges

    For example:

        >>> starts = [1, 3, 4, 6]
        >>> stops  = [1, 5, 7, 6]
        >>> vrange(starts, stops)
        array([3, 4, 4, 5, 6])

    """
    stops = np.asarray(stops)
    l = stops - starts # Lengths of each range.
    return np.repeat(stops - l.cumsum(), l) + np.arange(l.sum())
def latLongL2(original, secondary, k=4):
    o_lat = original[:,1]
    o_long = original[:,0]
    s_lat = secondary[:,1]
    s_long = secondary[:,0]
    
    diffs = (o_lat[:, None] - s_lat[None, :])**2 + (o_long[:, None] - s_long[None, :])**2
    indices = np.argpartition(diffs, k, axis=1)[:, :k]
    return indices
# ugly ones from me
def find_nearest(array, value): # finds closest value
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx
def square(list): # squares something in a list because it wasn't working for me
    return [i ** 2 for i in list]
def squared(list):
    return [i ** 1/2 for i in list]

MERRA2 INPUT VARIABLES: time, lon, lat, QV10M, T10M, TSKINWTR, U10, V10, EFLUXWTR, HFLUXWTR

In [151]:
# 2017 data import and organization

# let's do the entire thing by year. this way, we can just copy, paste, and adjust with year.
# also makes naming variables easier lol

sd = pd.read_csv('/Users/subhatrasivam/Documents/Internships/NOAA/Code/Saildrone/SD2017M2avghr.csv')
time_sd = sd['hours'].values
lon_sd = sd['lon'].values
lat_sd = sd['lat'].values

print('start MERRA2 import for 2017')

t0 = time.perf_counter()

filename_prod = '/Users/subhatrasivam/Documents/Internships/NOAA/Code/MERRA2/MERRA2_2017.nc'
f = nc.Dataset(filename_prod,mode='r')
time_prod = f.variables['time'][:] # hours since 05/01/2017 00:30:00
lon_prod = f.variables['lon'][:] # degrees
lat_prod = f.variables['lat'][:] # degrees
q_prod = f.variables['QV10M'][:] # kg/kg
airtemp_prod = f.variables['T10M'][:] # K
skntemp_prod = f.variables['TSKINWTR'][:] # K
uwind_prod = f.variables['U10M'][:] # m*s^-1
vwind_prod = f.variables['V10M'][:] # m*s^-1
eflux_prod = f.variables['EFLUXWTR'][:] # W*m^-2
hflux_prod = f.variables['HFLUXWTR'][:] # W*m^-2

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('completed MERRA2 2017 import: ' + delt +' s')

filepath = '/Users/subhatrasivam/Documents/Internships/NOAA/Code/MERRA2/2017MERRA2interp.csv'

start MERRA2 import for 2017
completed MERRA2 2017 import: 6.33 s


In [153]:
# 2018 data import and organization

sd = pd.read_csv('/Users/subhatrasivam/Documents/Internships/NOAA/Code/Saildrone/SD2018M2avghr.csv')
time_sd = sd['hours'].values
lon_sd = sd['lon'].values
lat_sd = sd['lat'].values

print('start MERRA2 import for 2018')

t0 = time.perf_counter()

filename_prod = '/Users/subhatrasivam/Documents/Internships/NOAA/Code/MERRA2/MERRA2_2018.nc'
f = nc.Dataset(filename_prod,mode='r')
time_prod = f.variables['time'][:] # hours since 05/01/2018 00:30:00
lon_prod = f.variables['lon'][:] # degrees
lat_prod = f.variables['lat'][:] # degrees
q_prod = f.variables['QV10M'][:] # kg/kg
airtemp_prod = f.variables['T10M'][:] # K
skntemp_prod = f.variables['TSKINWTR'][:] # K
uwind_prod = f.variables['U10M'][:] # m*s^-1
vwind_prod = f.variables['V10M'][:] # m*s^-1
eflux_prod = f.variables['EFLUXWTR'][:] # W*m^-2
hflux_prod = f.variables['HFLUXWTR'][:] # W*m^-2

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('completed MERRA2 2018 import: ' + delt +' s')

filepath = '/Users/subhatrasivam/Documents/Internships/NOAA/Code/MERRA2/2018MERRA2interp.csv'

start MERRA2 import for 2018
completed MERRA2 2018 import: 5.80 s


In [155]:
# 2019 data import and organization

sd = pd.read_csv('/Users/subhatrasivam/Documents/Internships/NOAA/Code/Saildrone/SD2019M2avghr.csv')
time_sd = sd['hours'].values
lon_sd = sd['lon'].values
lat_sd = sd['lat'].values

print('start MERRA2 import for 2019')

t0 = time.perf_counter()

filename_prod = '/Users/subhatrasivam/Documents/Internships/NOAA/Code/MERRA2/MERRA2_2019.nc'
f = nc.Dataset(filename_prod,mode='r')
time_prod = f.variables['time'][:] # hours since 05/01/2019 00:30:00
lon_prod = f.variables['lon'][:] # degrees
lat_prod = f.variables['lat'][:] # degrees
q_prod = f.variables['QV10M'][:] # kg/kg
airtemp_prod = f.variables['T10M'][:] # K
skntemp_prod = f.variables['TSKINWTR'][:] # K
uwind_prod = f.variables['U10M'][:] # m*s^-1
vwind_prod = f.variables['V10M'][:] # m*s^-1
eflux_prod = f.variables['EFLUXWTR'][:] # W*m^-2
hflux_prod = f.variables['HFLUXWTR'][:] # W*m^-2

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('completed MERRA2 2019 import: ' + delt +' s')

filepath = '/Users/subhatrasivam/Documents/Internships/NOAA/Code/MERRA2/2019MERRA2interp.csv'

start MERRA2 import for 2019
completed MERRA2 2019 import: 6.09 s


In [156]:
# interpolation and calculations

t0 = time.perf_counter()

# organize coordinates
location_sd = np.stack((lon_sd,lat_sd),axis=1) # set saildrone coordinates

gridlocation = [] # set MERRA2 coordinates
grididx = []
for idx_lon, lon in enumerate(lon_prod):
    for idx_lat,lat in enumerate(lat_prod):
        gridlocation.append([lon,lat])
        grididx.append([idx_lon,idx_lat])
gridlocation = np.asarray(gridlocation)
grididx = np.asarray(grididx)

close4 = latLongL2(location_sd,gridlocation)

# organize times
[idx_prod,idx_sd] = solution(time_prod,time_sd)
idx_prod = np.array(idx_prod)
timeidx_prod = np.copy(time_prod[idx_prod])
efluxidx_prod = np.copy(eflux_prod[idx_prod])
hfluxidx_prod = np.copy(hflux_prod[idx_prod])
airtempidx_prod = np.copy(airtemp_prod[idx_prod])
skntempidx_prod = np.copy(skntemp_prod[idx_prod])
qidx_prod = np.copy(q_prod[idx_prod])
uwindidx_prod = np.copy(uwind_prod[idx_prod])
vwindidx_prod = np.copy(vwind_prod[idx_prod])

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('organized times: ' + delt +' s')

t0 = time.perf_counter()

# get distance estimations for MERRA2 (haversine)
percentdist = []
for grid_idx, point in enumerate(location_sd):
    ind_close4 = close4[grid_idx]
    ind_gridlocations = []
    distancegrid = []
    for ind in ind_close4:
        gridpoint = gridlocation[ind]
        ind_gridlocations.append(gridpoint)
        lata, lona, latb, lonb, R = radians(gridpoint[1]),radians(gridpoint[0]), radians(point[1]), radians(point[0]), 6378.0
        lat_diff = latb-lata
        lon_diff = lonb-lona
        a = np.sin(lat_diff/2)**2+np.cos(latb)*np.cos(lata)*np.sin(lon_diff/2)**2
        c = 2*np.arctan2(a**(1/2),(1-a)**(1/2))
        distancepix = R * c
        distancegrid.append(np.divide(1,distancepix))
    sumdist = np.sum(distancegrid)
    percentdist.append(distancegrid/sumdist)

# get distance estimations for MERRA2 (pixel distance formula)
"""
percentdist = []
for grid_idx, point in enumerate(location_sd):
    ind_close4 = close4[grid_idx]
    ind_gridlocations = []
    distancegrid = []
    for ind in ind_close4:
        gridpoint = gridlocation[ind]
        ind_gridlocations.append(gridpoint)
        distancepix = ((point[0] - gridpoint[0])**2 + (point[1] - gridpoint[1])**2)**1/2
        distancegrid.append(np.divide(1,distancepix))
    sumdist = np.sum(distancegrid)
    percentdist.append(distancegrid/sumdist)
"""

percentdist = np.array(percentdist)

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('get inverse distance weights: ' + delt +' s')

t0 = time.perf_counter()

# interpolate MERRA2 variables
eflux_prod_withsd = []
hflux_prod_withsd = []
airtemp_prod_withsd = []
q_prod_withsd = []
skntemp_prod_withsd = []
uwind_prod_withsd = []
vwind_prod_withsd = []

for time_s in time_sd:
    for idx_time,time_m in enumerate(timeidx_prod):
        if time_s == time_m:
            eflux_prod_withsd.append(efluxidx_prod[idx_time])
            hflux_prod_withsd.append(hfluxidx_prod[idx_time])
            airtemp_prod_withsd.append(airtempidx_prod[idx_time])
            q_prod_withsd.append(qidx_prod[idx_time])
            skntemp_prod_withsd.append(skntempidx_prod[idx_time])
            uwind_prod_withsd.append(uwindidx_prod[idx_time])
            vwind_prod_withsd.append(vwindidx_prod[idx_time])
eflux_prod_withsd = np.array(eflux_prod_withsd)
hflux_prod_withsd = np.array(hflux_prod_withsd)
airtemp_prod_withsd = np.array(airtemp_prod_withsd)
q_prod_withsd = np.array(q_prod_withsd)
skntemp_prod_withsd = np.array(skntemp_prod_withsd)
uwind_prod_withsd = np.array(uwind_prod_withsd)
vwind_prod_withsd = np.array(vwind_prod_withsd)

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('get interpolated values: ' + delt +' s')

t0 = time.perf_counter()

# get 4 closest grid points
timeidx = np.arange(0,len(time_sd),1)
lat1 = []
lat2 = []
lat3 = []
lat4 = []
lon1 = []
lon2 = []
lon3 = []
lon4 = []

for k in close4:
    lat1.append(gridlocation[k[0]][1])
    lon1.append(gridlocation[k[0]][0])
    lat2.append(gridlocation[k[1]][1])
    lon2.append(gridlocation[k[1]][0])
    lat3.append(gridlocation[k[2]][1])
    lon3.append(gridlocation[k[2]][0])
    lat4.append(gridlocation[k[3]][1])
    lon4.append(gridlocation[k[3]][0])

arraylon1 = []
arraylon2 = []
arraylon3 = []
arraylon4 = []
arraylat1 = []
arraylat2 = []
arraylat3 = []
arraylat4 = []

for idx,lon in enumerate(lon1):
    arraylon1.append(find_nearest(lon_prod,lon))
    arraylon2.append(find_nearest(lon_prod,lon2[idx]))
    arraylon3.append(find_nearest(lon_prod,lon3[idx]))
    arraylon4.append(find_nearest(lon_prod,lon4[idx]))
    arraylat1.append(find_nearest(lat_prod,lat1[idx]))
    arraylat2.append(find_nearest(lat_prod,lat2[idx]))
    arraylat3.append(find_nearest(lat_prod,lat3[idx]))
    arraylat4.append(find_nearest(lat_prod,lat4[idx]))  

latlon1 = np.stack((arraylat1,arraylon1),axis=1)
latlon2 = np.stack((arraylat2,arraylon2),axis=1)   
latlon3 = np.stack((arraylat3,arraylon3),axis=1)   
latlon4 = np.stack((arraylat4,arraylon4),axis=1)

timeidx = np.arange(0,len(time_sd),1)

efluxlatlon1 = []
hfluxlatlon1 = []
qlatlon1 = []
airtemplatlon1 = []
skntemplatlot1 = []
uwindtemplatlot1 = []
vwindtemplatlot1 = []

efluxlatlon2 = []
hfluxlatlon2 = []
qlatlon2 = []
airtemplatlon2 = []
skntemplatlot2 = []
uwindtemplatlot2 = []
vwindtemplatlot2 = []

efluxlatlon3 = []
hfluxlatlon3 = []
qlatlon3 = []
airtemplatlon3 = []
skntemplatlot3 = []
uwindtemplatlot3 = []
vwindtemplatlot3 = []

efluxlatlon4 = []
hfluxlatlon4 = []
qlatlon4 = []
airtemplatlon4 = []
skntemplatlot4 = []
uwindtemplatlot4 = []
vwindtemplatlot4 = []

for j in timeidx:
    # first closest
    efluxlatlon1.append(eflux_prod_withsd[j,latlon1[j,0],latlon1[j,1]])
    hfluxlatlon1.append(hflux_prod_withsd[j,latlon1[j,0],latlon1[j,1]])
    qlatlon1.append(q_prod_withsd[j,latlon1[j,0],latlon1[j,1]])
    airtemplatlon1.append(airtemp_prod_withsd[j,latlon1[j,0],latlon1[j,1]])
    skntemplatlot1.append(skntemp_prod_withsd[j,latlon1[j,0],latlon1[j,1]])
    uwindtemplatlot1.append(uwind_prod_withsd[j,latlon1[j,0],latlon1[j,1]])
    vwindtemplatlot1.append(vwind_prod_withsd[j,latlon1[j,0],latlon1[j,1]])
    # second closest
    efluxlatlon2.append(eflux_prod_withsd[j,latlon2[j,0],latlon2[j,1]])
    hfluxlatlon2.append(hflux_prod_withsd[j,latlon2[j,0],latlon2[j,1]])
    qlatlon2.append(q_prod_withsd[j,latlon2[j,0],latlon2[j,1]])
    airtemplatlon2.append(airtemp_prod_withsd[j,latlon2[j,0],latlon2[j,1]])
    skntemplatlot2.append(skntemp_prod_withsd[j,latlon2[j,0],latlon2[j,1]])
    uwindtemplatlot2.append(uwind_prod_withsd[j,latlon2[j,0],latlon2[j,1]])
    vwindtemplatlot2.append(vwind_prod_withsd[j,latlon2[j,0],latlon2[j,1]])
    # third closest
    efluxlatlon3.append(eflux_prod_withsd[j,latlon3[j,0],latlon3[j,1]])
    hfluxlatlon3.append(hflux_prod_withsd[j,latlon3[j,0],latlon3[j,1]])
    qlatlon3.append(q_prod_withsd[j,latlon3[j,0],latlon3[j,1]])
    airtemplatlon3.append(airtemp_prod_withsd[j,latlon3[j,0],latlon3[j,1]])
    skntemplatlot3.append(skntemp_prod_withsd[j,latlon3[j,0],latlon3[j,1]])
    uwindtemplatlot3.append(uwind_prod_withsd[j,latlon3[j,0],latlon3[j,1]])
    vwindtemplatlot3.append(vwind_prod_withsd[j,latlon3[j,0],latlon3[j,1]])
    # furthest
    efluxlatlon4.append(eflux_prod_withsd[j,latlon4[j,0],latlon4[j,1]])
    hfluxlatlon4.append(hflux_prod_withsd[j,latlon4[j,0],latlon4[j,1]])
    qlatlon4.append(q_prod_withsd[j,latlon4[j,0],latlon4[j,1]])
    airtemplatlon4.append(airtemp_prod_withsd[j,latlon4[j,0],latlon4[j,1]])
    skntemplatlot4.append(skntemp_prod_withsd[j,latlon4[j,0],latlon4[j,1]])
    uwindtemplatlot4.append(uwind_prod_withsd[j,latlon4[j,0],latlon4[j,1]])
    vwindtemplatlot4.append(vwind_prod_withsd[j,latlon4[j,0],latlon4[j,1]])

for_df = np.array([timeidx,efluxlatlon1,hfluxlatlon1,airtemplatlon1,skntemplatlot1,qlatlon1,uwindtemplatlot1,vwindtemplatlot1,percentdist[:,0]])
for_df = for_df.T
for_df = np.ma.masked_where(for_df == 999999986991104, for_df)
for_df = np.ma.compress_rows(for_df)
df = pd.DataFrame(for_df, columns=['time','eflux','hflux','airtemp','skntemp','q','uwind','vwind','percent'])

time1 = df['time']
efluxlatlon1 = df['eflux'].values
hfluxlatlon1 = df['hflux'].values
airtemplatlon1 = df['airtemp'].values
skntemplatlot1 = df['skntemp'].values
qlatlon1 = df['q'].values
uwindtemplatlot1 = df['uwind'].values
vwindtemplatlot1 = df['vwind'].values
percent1 = df['percent'].values

efluxlatlon1 = np.array(efluxlatlon1)
hfluxlatlon1 = np.array(hfluxlatlon1)
qlatlon1 = np.array(qlatlon1)
airtemplatlon1 = np.array(airtemplatlon1)
skntemplatlot1 = np.array(skntemplatlot1)
uwindtemplatlot1 = np.array(uwindtemplatlot1)
vwindtemplatlot1 = np.array(vwindtemplatlot1)
percent1 = np.array(percent1)

for_df = np.array([timeidx,efluxlatlon2,hfluxlatlon2,airtemplatlon2,skntemplatlot2,qlatlon2,uwindtemplatlot2,vwindtemplatlot2,percentdist[:,1]])
for_df = for_df.T
for_df = np.ma.masked_where(for_df == 999999986991104, for_df)
for_df = np.ma.compress_rows(for_df)
df = pd.DataFrame(for_df, columns=['time','eflux','hflux','airtemp','skntemp','q','uwind','vwind','percent'])

time2 = df['time']
efluxlatlon2 = df['eflux'].values
hfluxlatlon2 = df['hflux'].values
airtemplatlon2 = df['airtemp'].values
skntemplatlot2 = df['skntemp'].values
qlatlon2 = df['q'].values
uwindtemplatlot2 = df['uwind'].values
vwindtemplatlot2 = df['vwind'].values
percent2 = df['percent'].values

efluxlatlon2 = np.array(efluxlatlon2)
hfluxlatlon2 = np.array(hfluxlatlon2)
qlatlon2 = np.array(qlatlon2)
airtemplatlon2 = np.array(airtemplatlon2)
skntemplatlot2 = np.array(skntemplatlot2)
uwindtemplatlot2 = np.array(uwindtemplatlot2)
vwindtemplatlot2 = np.array(vwindtemplatlot2)
percent2 = np.array(percent2)

for_df = np.array([timeidx,efluxlatlon3,hfluxlatlon3,airtemplatlon3,skntemplatlot3,qlatlon3,uwindtemplatlot3,vwindtemplatlot3,percentdist[:,2]])
for_df = for_df.T
for_df = np.ma.masked_where(for_df == 999999986991104, for_df)
for_df = np.ma.compress_rows(for_df)
df = pd.DataFrame(for_df, columns=['time','eflux','hflux','airtemp','skntemp','q','uwind','vwind','percent'])

time3 = df['time']
efluxlatlon3 = df['eflux'].values
hfluxlatlon3 = df['hflux'].values
airtemplatlon3 = df['airtemp'].values
skntemplatlot3 = df['skntemp'].values
qlatlon3 = df['q'].values
uwindtemplatlot3 = df['uwind'].values
vwindtemplatlot3 = df['vwind'].values
percent3 = df['percent'].values

efluxlatlon3 = np.array(efluxlatlon3)
hfluxlatlon3 = np.array(hfluxlatlon3)
qlatlon3 = np.array(qlatlon3)
airtemplatlon3 = np.array(airtemplatlon3)
skntemplatlot3 = np.array(skntemplatlot3)
uwindtemplatlot3 = np.array(uwindtemplatlot3)
vwindtemplatlot3 = np.array(vwindtemplatlot3)
percent3 = np.array(percent3)

for_df = np.array([time_sd,efluxlatlon4,hfluxlatlon4,airtemplatlon4,skntemplatlot4,qlatlon4,uwindtemplatlot4,vwindtemplatlot4,percentdist[:,3]])
for_df = for_df.T
for_df = np.ma.masked_where(for_df == 999999986991104, for_df)
for_df = np.ma.compress_rows(for_df)
df = pd.DataFrame(for_df, columns=['time','eflux','hflux','airtemp','skntemp','q','uwind','vwind','percent'])

time_sd = df['time']
efluxlatlon4 = df['eflux'].values
hfluxlatlon4 = df['hflux'].values
airtemplatlon4 = df['airtemp'].values
skntemplatlot4 = df['skntemp'].values
qlatlon4 = df['q'].values
uwindtemplatlot4 = df['uwind'].values
vwindtemplatlot4 = df['vwind'].values
percent4 = df['percent'].values

efluxlatlon4 = np.array(efluxlatlon4)
hfluxlatlon4 = np.array(hfluxlatlon4)
qlatlon4 = np.array(qlatlon4)
airtemplatlon4 = np.array(airtemplatlon4)
skntemplatlot4 = np.array(skntemplatlot4)
uwindtemplatlot4 = np.array(uwindtemplatlot4)
vwindtemplatlot4 = np.array(vwindtemplatlot4)
percent4 = np.array(percent4)

# next step is to get rid of everything(!) that has a NAN values for all four values. makes sense in my head sorry to whoever is reading this
# we already got rid of NAN valules per the four, but now it is a matter of getting rid of all of it
# ok we are actually good on time for 2017!!! this may be a coincidence and we need to fix this for ERA5. but i checked the data and we are good baby

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('get 4 closest variable values: ' + delt +' s')

# get weighted average
eflux = efluxlatlon1*percent1 + efluxlatlon2*percent2 + efluxlatlon3*percent3 + efluxlatlon4*percent4
hflux = hfluxlatlon1*percent1 + hfluxlatlon2*percent2 + hfluxlatlon3*percent3 + hfluxlatlon4*percent4
airtemp = airtemplatlon1*percent1 + airtemplatlon2*percent2 + airtemplatlon3*percent3 + airtemplatlon4*percent4
skntemp = skntemplatlot1*percent1 + skntemplatlot2*percent2 + skntemplatlot3*percent3 + skntemplatlot4*percent4
q = qlatlon1*percent1 + qlatlon2*percent2 + qlatlon3*percent3 + qlatlon4*percent4
uwind = uwindtemplatlot1*percent1 + uwindtemplatlot2*percent2 + uwindtemplatlot3*percent3 + uwindtemplatlot4*percent4
vwind = vwindtemplatlot1*percent1 + vwindtemplatlot2*percent2 + vwindtemplatlot3*percent3 + vwindtemplatlot4*percent4

t0 = time.perf_counter()

# calculate variables
airtemp = np.array(np.subtract(airtemp,273.15)) # K -> C
skntemp = np.array(np.subtract(skntemp,273.15)) # K -> C

def square(list):
    return [i ** 2 for i in list]
def squared(list):
    return [i ** 1/2 for i in list]
usq = square(uwind)
vsq = square(vwind)
sum  = np.add(usq,vsq)
windidx_prod = squared(sum) # u and v -> total magnitude
wind = np.array(windidx_prod)

es_tempfin_prod = 6.11*2.71828**(5420*(1/273-1/(airtemp+273.15))) # air saturation vapor pressure OK
es_sknfin_prod = 6.11*2.71828**(5420*(1/273-1/(skntemp+273.15))) # sea saturation vapor pressure OK

rat_prh = 0.622*es_tempfin_prod*((q+1)/q) #mb/rh dec

qskn = 1000*311*es_sknfin_prod/(500*rat_prh-311*es_sknfin_prod)
qair = np.multiply(q,1000) # kg/kg -> g/kg

t1 = time.perf_counter()
delt = str("{:.2f}".format(t1-t0))

print('calculated variables: ' + delt +' s')

for_df = np.stack([time_sd,eflux,hflux,qair,qskn,airtemp,skntemp,wind],axis=1)
df = pd.DataFrame(for_df,columns=['time','eflux','hflux','qair','qskn','airtemp','skntemp','wind'])
df.to_csv(filepath)

organized times: 4.08 s
get inverse distance weights: 1.01 s
get interpolated values: 15.05 s
get 4 closest variable values: 2.83 s
calculated variables: 0.02 s
